<a href="https://colab.research.google.com/github/TainaDr/pipeline_titanic/blob/main/etl_aula1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Realize a extração dos dados do Titanic



In [ ]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/TainaDr/pipeline_titanic/main/titanic.csv"
dataFrame = pd.read_csv(url)

## 2. Exibição das informações do conjunto de dados

In [ ]:
print(dataFrame.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


## 3. Exibição das estatísticas descritivas

In [ ]:
print(dataFrame.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


## 4. Demonstração dos valores ausentes por coluna

In [ ]:
dataFrame.columns = dataFrame.columns.str.strip()
print(dataFrame.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


## 5. Age: Preenchimento das idades faltantes utilizando a mediana por classe e sexo

In [ ]:
def preencher_idade(row):
    if pd.isna(row['Age']):
        mediana = dataFrame.groupby(['Pclass', 'Sex'])['Age'].median()
        return mediana.loc[row['Pclass'], row['Sex']]
    else:
        return row['Age']

dataFrame['Age'] = dataFrame.apply(preencher_idade, axis=1)

## 6. Age: Preenchimento dos valores ausentes restantes com a mediana geral

In [ ]:
medianaGeral = dataFrame['Age'].median()
dataFrame['Age'].fillna(mediana)

,Age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0
...,...
886,27.0
887,19.0
888,28.0
889,26.0


## 7. Embarked: Preenchimento dos valores ausentes com o valor mais frequente

In [ ]:
mais_frequente = dataFrame['Embarked'].mode()[0]
dataFrame['Embarked'] = dataFrame['Embarked'].fillna(mais_frequente)

## 8. Cabin: Criação de feature indicadora se existe cabine ou não

In [ ]:
dataFrame[''] = dataFrame['Cabin'].notna().astype(int)
print(dataFrame[['Cabin', 'HasCabin']])

    Cabin  HasCabin
0     NaN         0
1     C85         1
2     NaN         0
3    C123         1
4     NaN         0
..    ...       ...
886   NaN         0
887   B42         1
888   NaN         0
889  C148         1
890   NaN         0

[891 rows x 2 columns]


## 9. Fare: Preenchimento dos valores ausentes com a média, agrupando por classe

In [ ]:
dataFrame['Age'] = dataFrame['Age'].fillna(
    dataFrame.groupby('Pclass')['Age'].transform('mean')
)

print("Valores ausentes em Age:", dataFrame['Age'].isna().sum())

print(dataFrame[['Pclass', 'Age']])

Valores ausentes em Age: 0
     Pclass   Age
0         3  22.0
1         1  38.0
2         3  26.0
3         1  35.0
4         3  35.0
..      ...   ...
886       2  27.0
887       1  19.0
888       3  28.0
889       1  26.0
890       3  32.0

[891 rows x 2 columns]


## 10. Age: Criação de feature indicativa para grupos de idade

In [ ]:
dataFrame['grupoIdade'] = pd.cut(
    dataFrame["Age"],
    bins = [0, 12, 18, 35, 60, 100],
    labels = ['Criança', 'Adolescente', 'Jovem', 'Adulto', 'Idoso']
)

## 11. Fare: Criação de feature indicativa para grupos de tarifa

In [ ]:
dataFrame['grupoTarifa'] = pd.cut(
    dataFrame["Fare"],
    bins = [0, 7.9, 14.45, 31, 600],
    labels = ['Baixa', 'Média', 'Média-Alta', 'Alta']
)

In [ ]:
dataFrame.to_csv('corrigido.csv')